## Sequence feature analysis
Here, I want to conduct an in-depth analysis of the sequences in which the previous three models were all right & wrong, and investigate the reasons for these sequences being wrong

### 1. Convert the.pt file to.fasta file first

In [ ]:
import torch
import os

def save_sequences_to_fasta(pt_file, output_fasta):
    data = torch.load(pt_file)
    
    if 'gene_id' not in data or 'sequences' not in data:
        print(f"File {pt_file} does not contain 'gene_id' or 'sequences'.")
        return
    
    with open(output_fasta, 'w', encoding='utf-8') as f:
        for gene_id, sequence in zip(data['gene_id'], data['sequences']):
            f.write(f">{gene_id}\n")
            # Wrap the sequence by 60 characters to ensure that long sequences do not break the format
            for i in range(0, len(sequence), 60):
                f.write(sequence[i:i+60] + "\n")
    print(f"Saved sequences to {output_fasta} in FASTA format.")

output_dir = "3model_result"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
correct_pt_file = os.path.join(output_dir, "2model_all_correct_nt_cd.pt")
wrong_pt_file = os.path.join(output_dir, "2model_all_wrong_nt_cd.pt")
correct_fasta_file = os.path.join(output_dir, "2model_all_correct_nt_cd.fasta")
wrong_fasta_file = os.path.join(output_dir, "2model_all_wrong_nt_cd.fasta")

# Save the correct sequence of all model predictions to a FASTA file
save_sequences_to_fasta(correct_pt_file, correct_fasta_file)

save_sequences_to_fasta(wrong_pt_file, wrong_fasta_file)

Saved sequences to 3model_result/2model_all_correct_nt_cd.fasta in FASTA format.
Saved sequences to 3model_result/2model_all_wrong_nt_cd.fasta in FASTA format.


/var/folders/69/y_h_2twd02q4y_zk07g55xk40000gp/T/ipykernel_20430/3313765355.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(pt_file)


Sort out the files that do one right and do two right

In [ ]:
correct_1_file = os.path.join(output_dir, "3model_have_1_correct.pt")
correct_1_fasta_file = os.path.join(output_dir, "3model_have_1_correct.fasta")

correct_2_pt_file = os.path.join(output_dir, "3model_have_2_correct.pt")
correct_2_fasta_file = os.path.join(output_dir, "3model_have_2_correct.fasta")

save_sequences_to_fasta(correct_1_file, correct_1_fasta_file)

save_sequences_to_fasta(correct_2_pt_file, correct_2_fasta_file)

Saved sequences to 3model_result/3model_have_1_correct.fasta in FASTA format.
Saved sequences to 3model_result/3model_have_2_correct.fasta in FASTA format.


/var/folders/69/y_h_2twd02q4y_zk07g55xk40000gp/T/ipykernel_69738/326448979.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(pt_file)


### 2. Sequence comparison
Compare the features of different types of sequences horizontally to find out if there are any differences

In [ ]:
from Bio import SeqIO
from Bio.SeqUtils import gc_fraction
from collections import Counter
import pandas as pd

def extract_features_from_sequence(sequence):
    # Base composition
    base_counts = Counter(sequence)
    total_bases = len(sequence)
    base_composition = {base: base_counts.get(base, 0) / total_bases for base in 'ATCG'}
    
    # Diplet frequency
    dinucleotides = [sequence[i:i+2] for i in range(len(sequence)-1)]
    dinuc_counts = Counter(dinucleotides)
    dinuc_freq = {k: dinuc_counts.get(k, 0) / (len(sequence) - 1) for k in [a + b for a in 'ATCG' for b in 'ATCG']}
    
    # Triplet frequency
    trinucleotides = [sequence[i:i+3] for i in range(len(sequence)-2)]
    trinuc_counts = Counter(trinucleotides)
    trinuc_freq = {k: trinuc_counts.get(k, 0) / (len(sequence) - 2) for k in [a + b + c for a in 'ATCG' for b in 'ATCG' for c in 'ATCG']}
    
    # GC content
    gc_content = gc_fraction(sequence)
    
    features = {
        **base_composition,
        **dinuc_freq,
        **trinuc_freq,
        'gc_content': gc_content,
        'length': total_bases
    }
    
    return features

def load_fasta_and_extract_features(fasta_file):
    features = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        seq = str(record.seq)
        features.append(extract_features_from_sequence(seq))
    return pd.DataFrame(features)

In [ ]:
wrong_features_df = load_fasta_and_extract_features("3model_result/3model_all_wrong.fasta")
correct_1_features_df = load_fasta_and_extract_features("3model_result/3model_have_1_correct.fasta")
correct_2_features_df = load_fasta_and_extract_features("3model_result/3model_have_2_correct.fasta")
correct_features_df = load_fasta_and_extract_features("3model_result/3model_all_correct.fasta")


In [13]:
wrong_features_df

,A,T,C,G,AA,AT,AC,AG,TA,TT,...,GCA,GCT,GCC,GCG,GGA,GGT,GGC,GGG,gc_content,length
0,0.31495,0.28485,0.20185,0.19835,0.114706,0.078304,0.054003,0.067903,0.070604,0.097555,...,0.013351,0.013451,0.013201,0.003800,0.015802,0.011401,0.012401,0.011701,0.40020,20000
1,0.27030,0.27075,0.22395,0.23500,0.082854,0.062453,0.048302,0.076654,0.051753,0.083204,...,0.017802,0.017202,0.013351,0.003400,0.019002,0.013501,0.017402,0.021952,0.45895,20000
2,0.31685,0.26815,0.19910,0.21590,0.112806,0.073904,0.053553,0.076604,0.065603,0.078954,...,0.015002,0.013751,0.011701,0.003250,0.016802,0.012701,0.013451,0.013001,0.41500,20000
3,0.24140,0.23780,0.25465,0.26615,0.070304,0.042152,0.046752,0.082204,0.034202,0.062253,...,0.018452,0.020802,0.021752,0.005351,0.020402,0.015802,0.023052,0.024452,0.52080,20000
4,0.26735,0.27070,0.22590,0.23605,0.082604,0.063653,0.048602,0.072504,0.054103,0.086054,...,0.016002,0.016552,0.018802,0.006851,0.018002,0.013401,0.017202,0.018352,0.46195,20000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,0.18940,0.20570,0.27835,0.32655,0.034852,0.030302,0.039552,0.084704,0.019651,0.045052,...,0.021802,0.022102,0.031953,0.010151,0.026353,0.019102,0.033253,0.047355,0.60490,20000
73,0.23240,0.23340,0.25885,0.27535,0.057103,0.040302,0.047952,0.087004,0.032502,0.062053,...,0.018202,0.020252,0.022252,0.005851,0.022452,0.017252,0.022652,0.028153,0.53420,20000
74,0.26660,0.23190,0.26260,0.23890,0.079504,0.049352,0.057753,0.080004,0.041902,0.059703,...,0.018602,0.018352,0.022302,0.005001,0.018052,0.012901,0.021752,0.017152,0.50150,20000
75,0.26765,0.28005,0.21375,0.23855,0.081404,0.066653,0.046702,0.072854,0.060053,0.091955,...,0.015802,0.016202,0.016902,0.006751,0.018152,0.013551,0.017152,0.020402,0.45230,20000


In [14]:
correct_1_features_df

,A,T,C,G,AA,AT,AC,AG,TA,TT,...,GCA,GCT,GCC,GCG,GGA,GGT,GGC,GGG,gc_content,length
0,0.23925,0.28325,0.23910,0.23840,0.065603,0.060203,0.044352,0.069103,0.046702,0.089054,...,0.014751,0.018202,0.017102,0.005601,0.017852,0.013901,0.016852,0.021002,0.47750,20000
1,0.27985,0.28875,0.20970,0.22170,0.093705,0.067503,0.049152,0.069503,0.061003,0.096105,...,0.014501,0.015452,0.015202,0.005651,0.014101,0.014251,0.016652,0.014001,0.43140,20000
2,0.26845,0.25755,0.24005,0.23395,0.090155,0.058153,0.050003,0.070104,0.052203,0.081504,...,0.015402,0.017152,0.021352,0.010201,0.015852,0.013151,0.020352,0.016852,0.47400,20000
3,0.29060,0.28175,0.20520,0.22245,0.096355,0.073704,0.049402,0.071104,0.062453,0.091155,...,0.014701,0.013551,0.014551,0.005101,0.017652,0.012551,0.014151,0.017502,0.42765,20000
4,0.30495,0.28970,0.19310,0.21225,0.105205,0.079604,0.048602,0.071504,0.066403,0.095605,...,0.013801,0.013351,0.011651,0.003350,0.016952,0.011551,0.012401,0.014801,0.40535,20000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,0.29780,0.29470,0.20400,0.20350,0.099405,0.073654,0.051703,0.073004,0.064103,0.097505,...,0.014101,0.014551,0.011551,0.002650,0.015352,0.010151,0.011451,0.010751,0.40750,20000
77,0.29940,0.27240,0.21100,0.21720,0.106055,0.069253,0.049802,0.074304,0.062903,0.087304,...,0.015152,0.014301,0.014801,0.004550,0.017502,0.012151,0.014551,0.014201,0.42820,20000
78,0.32695,0.31015,0.17360,0.18930,0.118806,0.092455,0.050153,0.065553,0.081654,0.107305,...,0.013001,0.011601,0.007451,0.001850,0.014901,0.010451,0.009001,0.010701,0.36290,20000
79,0.22610,0.24600,0.25915,0.26875,0.059353,0.044902,0.047452,0.074404,0.035952,0.062353,...,0.018152,0.019652,0.023552,0.006001,0.019452,0.016352,0.022752,0.026903,0.52790,20000


In [15]:
correct_2_features_df

,A,T,C,G,AA,AT,AC,AG,TA,TT,...,GCA,GCT,GCC,GCG,GGA,GGT,GGC,GGG,gc_content,length
0,0.28155,0.31725,0.19745,0.20375,0.091855,0.077154,0.047052,0.065503,0.064503,0.113506,...,0.012951,0.013501,0.011301,0.002500,0.015002,0.012601,0.011701,0.011851,0.40120,20000
1,0.29885,0.26500,0.21470,0.22145,0.107305,0.066103,0.052903,0.072554,0.059603,0.086604,...,0.014851,0.012301,0.013251,0.008501,0.017952,0.011501,0.013601,0.018202,0.43615,20000
2,0.32620,0.25495,0.21300,0.20585,0.118256,0.072704,0.058003,0.077204,0.060603,0.075754,...,0.016402,0.011901,0.011351,0.003900,0.018302,0.010401,0.011351,0.013201,0.41885,20000
3,0.24725,0.27610,0.23715,0.23950,0.074054,0.055503,0.046052,0.071654,0.043202,0.089554,...,0.016202,0.018302,0.019102,0.005001,0.017952,0.015152,0.018652,0.017502,0.47665,20000
4,0.27185,0.29700,0.22340,0.20775,0.085204,0.069353,0.049902,0.067403,0.060753,0.099805,...,0.014001,0.014101,0.015902,0.003500,0.013851,0.011951,0.013651,0.013201,0.43115,20000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,0.28350,0.31350,0.19690,0.20610,0.091705,0.080004,0.044052,0.067703,0.069853,0.116256,...,0.016552,0.014151,0.011451,0.006201,0.013851,0.010901,0.011451,0.011501,0.40300,20000
155,0.20550,0.20805,0.28790,0.29855,0.048402,0.031502,0.045802,0.079804,0.024001,0.049702,...,0.019602,0.021452,0.031453,0.010101,0.022102,0.017452,0.030103,0.035704,0.58645,20000
156,0.26715,0.32555,0.19675,0.21055,0.079554,0.076654,0.042652,0.068253,0.067853,0.117256,...,0.012951,0.016252,0.010301,0.002850,0.013501,0.013501,0.012001,0.012701,0.40730,20000
157,0.28505,0.28560,0.21685,0.21250,0.090905,0.071754,0.052553,0.069853,0.061653,0.092455,...,0.014151,0.014601,0.014451,0.004550,0.015052,0.012951,0.014151,0.012751,0.42935,20000


In [6]:
correct_features_df

,A,T,C,G,AA,AT,AC,AG,TA,TT,...,GCA,GCT,GCC,GCG,GGA,GGT,GGC,GGG,gc_content,length
0,0.24115,0.21785,0.26040,0.28060,0.069203,0.039802,0.050153,0.082004,0.029501,0.052603,...,0.018652,0.019152,0.023002,0.008751,0.023752,0.017652,0.024902,0.029303,0.54100,20000
1,0.33180,0.32555,0.17420,0.16845,0.122156,0.096055,0.049452,0.064153,0.087854,0.115206,...,0.011951,0.010851,0.007801,0.000850,0.011301,0.008951,0.007201,0.006301,0.34265,20000
2,0.26830,0.27455,0.22615,0.23100,0.081754,0.062803,0.050453,0.073254,0.057653,0.088004,...,0.016002,0.017052,0.017052,0.005451,0.016952,0.014201,0.017102,0.016202,0.45715,20000
3,0.26300,0.30535,0.21310,0.21855,0.077754,0.069153,0.047202,0.068903,0.065603,0.106505,...,0.013251,0.016152,0.015402,0.006601,0.015052,0.013301,0.014701,0.013301,0.43165,20000
4,0.30705,0.30385,0.19390,0.19520,0.100255,0.086004,0.049752,0.071054,0.073654,0.099205,...,0.013451,0.012101,0.009551,0.001500,0.014551,0.010751,0.009551,0.011751,0.38910,20000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,0.27430,0.34340,0.18595,0.19635,0.085854,0.083254,0.042402,0.062803,0.074004,0.132257,...,0.011601,0.013251,0.010251,0.002850,0.012601,0.012751,0.010301,0.009601,0.38230,20000
669,0.23940,0.24930,0.23925,0.27205,0.061203,0.048152,0.045602,0.084454,0.042552,0.065653,...,0.016202,0.020052,0.019052,0.005301,0.021552,0.017052,0.020802,0.027553,0.51130,20000
670,0.28425,0.28230,0.20625,0.22720,0.090655,0.071254,0.047652,0.074704,0.060403,0.087954,...,0.014451,0.015152,0.014201,0.004450,0.017802,0.013951,0.013251,0.015702,0.43345,20000
671,0.26635,0.31665,0.22135,0.19565,0.080604,0.075154,0.048202,0.062403,0.061853,0.105405,...,0.013051,0.017402,0.012551,0.001400,0.012451,0.012151,0.012451,0.012101,0.41700,20000
